**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Análisis de sentimiento a partir de comentarios de películas

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Referencias

1. [Sentiment Analysis guide](https://medium.com/seek-blog/your-guide-to-sentiment-analysis-344d43d225a7)
2. [LSTM-sentiment-analysis-with-tensorflow-keras](https://github.com/paulkarikari/LSTM-sentiment-analysis-with-tensorflow-keras-api/blob/master/sentiment_analysis_tweets.ipynb)

In [2]:
# pip install sentiment-analysis-spanish

## Contenido

* [Introducción](#Introducción)
* [Datos](#Datos)
* [Preprocesamiento](#Preprocesamiento)
* [Modelo Keras](#Modelo-Keras)
* [Datos para pruebas](#Datos-para-pruebas)

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.1.0


## Introducción

Gran conjunto de datos de revisión de películas.

Este es un conjunto de datos para la clasificación de sentimiento binario que contiene sustancialmente más datos que los conjuntos de datos de referencia anteriores. 

Es un conjunto de 25.000 reseñas de películas muy polares para entrenamiento y 25.000 para pruebas. También hay datos adicionales sin etiquetar para su uso. 

## Datos

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

## Preprocesamiento

In [5]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


## Modelo Keras

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 10s 394us/sample - loss: 0.4970 - accuracy: 0.7386 - val_loss: 0.3443 - val_accuracy: 0.8490
Epoch 2/10
25000/25000 [==============================] - 8s 336us/sample - loss: 0.2335 - accuracy: 0.9086 - val_loss: 0.3662 - val_accuracy: 0.8412
Epoch 3/10
25000/25000 [==============================] - 9s 343us/sample - loss: 0.0803 - accuracy: 0.9807 - val_loss: 0.4653 - val_accuracy: 0.8250
Epoch 4/10
25000/25000 [==============================] - 9s 342us/sample - loss: 0.0191 - accuracy: 0.9975 - val_loss: 0.5368 - val_accuracy: 0.8264
Epoch 5/10
25000/25000 [==============================] - 9s 340us/sample - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.5929 - val_accuracy: 0.8276
Epoch 6/10
25000/25000 [==============================] - 9s 362us/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6521 - val_accuracy: 0.8260
Epoch 7/10
25000/25000 [=========================

In [10]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


## Datos para pruebas

In [11]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [12]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [15]:
def predict(sentence_list):
    sequence_list = tokenizer.texts_to_sequences(sentence_list)
    sequence_list = pad_sequences(sequence_list, maxlen=max_length, truncating=trunc_type)
    return model.predict(sequence_list)

comments = []
comments.append("I really think this is amazing. it is  good, honest.")
comments.append("it is really boring. Is it is very bad")

print(predict( comments))



[[0.99701416]
 [0.00426486]]
